# Chainerチュートリアル
まずはコマンド・プロンプトないしそれ相当のもので"pip install chainer"

In [1]:
import chainer
import numpy as np
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [2]:
x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)

Variable型はChainer独自の型で変数を格納できるみたい

In [3]:
print(x_data)
print(x)

[ 5.]
<var@1be96bb7c50>


ここからはVariableの性質を見ていく

In [4]:
y = x**2 - 2*x + 1

In [5]:
y

中身はそのままだと見れない

In [6]:
print(y)

<var@1be96bb7a90>


In [7]:
y.data

array([ 16.], dtype=float32)

In [8]:
print(y.data)

[ 16.]


これで見れる。値はdataに格納される。ChainerのVariable型は値の他に履歴や便利な機能を提供するために直接的には値が参照できない。その便利な機能の一つが以下のbackward()

In [9]:
print(y.backward())

None


別段backward()そのものは何も返さないが…

In [10]:
print(x.grad)

[ 8.]


In [11]:
x.grad

array([ 8.], dtype=float32)

x.gradに変化がある。これはつまりyのxに関する勾配が格納されている。今回ならdy/dx = 2x -2, x = 5なので 8で確かに正しい。

In [12]:
y.backward()

In [13]:
print(x.grad)

[ 16.]


もう一度backwardをすると値が累積するみたいだ。これはニューラルネットで実際にバックプロパゲーションするときにバッチ処理（たくさんのデータをまとめて処理すること）するのに便利だからだろう。

In [14]:
x.grad = np.array([0], dtype=np.float32)
print(x.grad)
y.backward()
print(x.grad)

[ 0.]
[ 8.]


一応値はリセットできるみたいだ。ただし注意が必要で、ここで初めて気づいけど、どうやらChainerは基本の型をnumpyのnp.float32型としているらしく代入する値をこれ以外にするとエラーを吐く。

次は中間に変数がある場合について見てみる。

In [15]:
x_data = np.array([5.], dtype=np.float32)
x = Variable(x_data)
z = 2*x
y = x**2 - z + 1
print("retain_grad = True")
y.backward(retain_grad=True)
print("zgrad:",z.grad, ", z:", z.data)
print("xgrad:",x.grad, ", x:", x.data)
print("retain_grad = False")
x = Variable(x_data)
z = 2*x
y = x**2 - z + 1
y.backward(retain_grad=False)
print("zgrad:",z.grad, ", z:", z.data)
print("xgrad:",x.grad, ", x:", x.data)

retain_grad = True
zgrad: [-1.] , z: [ 10.]
xgrad: [ 8.] , x: [ 5.]
retain_grad = False
zgrad: None , z: [ 10.]
xgrad: [ 8.] , x: [ 5.]


retain_grad をTrueにするとｚのgradにも値が格納される。あまり使うことは無いらしい。また、retain_gradの真偽に関わらず、ｘのgradはｚを展開した式の形になる。

正直この方法だと沢山のデータの逆伝播を計算したいときは不便だ。そこで、ｘには行列が代入できる

In [37]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
y = x**2 - 2*x + 1
y.grad = np.ones((2, 3), dtype=np.float32) * 2
y.backward()
x.grad

array([[  0.,   4.,   8.],
       [ 12.,  16.,  20.]], dtype=float32)

ここでy.gradに値を代入してるのは初期誤差値（ｙをｘから予想された値だとすると予想と事実の差）だそう。要するにy.grad * dy/dx が　x.gradとなる

ChainerのFunctionモジュールの関数はVariable型を引数にとる物が多いらしく、これらを組み合わせることで複雑なバックプロパゲーション（backward()のこと）の計算ができるとのこと。

# Linksを使った表現

ニューラルネットでは素子と素子とを「繋げる」ということがとても大切になってくる。もちろんChainerではその動作が簡単に書けるのだ。  
次に書くのは最もよく使われる線形変換で3次元の配列を2次元に変換するための線形変換（アファイン変換、全結合）だ。

In [17]:
f = L.Linear(3, 2)

(L.linear と　L.Linear どっちもあるって罠だな。)  
この関数ｆは3次元の引数を取るが、ほとんどのChainerの関数同様、パッチ入力しか受け取らない。要するに、（N、3）の行列を受け取る。このNがミニバッチのサイズ

これで f(x) = Wx + b (勿論Wは行列、bはベクトルだぞ)　の式を作ることができた。与えた引数によってWは　2* 3,　 b は2の長さになっているぞ  
(個人的な注意）今までｘがパラメータだったがここからはｘはパラメータでは無くデータ。パラメータ（変更すべき係数）はWとｂ。

In [18]:
print("W:",f.W.data)
print("b:",f.b.data)

W: [[-0.64312094  0.49359825  0.36686105]
 [ 0.57213914 -0.91328198 -0.10738265]]
b: [ 0.  0.]


Wはランダムな値、bは０で初期化されている

In [19]:
f.W = Variable(np.array([[1,0,0],[0,1,0]], dtype=np.float32))
f.b = Variable(np.array([1,1], dtype=np.float32))

In [20]:
print("W:",f.W.data)
print("b:",f.b.data)

W: [[ 1.  0.  0.]
 [ 0.  1.  0.]]
b: [ 1.  1.]


実験のために値をわかりやすく指定した。

つぎにミニバッチ数２のデータを適当に突っ込む。

In [42]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float32))
y = f(x)
y.data

array([[ 2.,  3.],
       [ 5.,  6.],
       [ 8.,  9.]], dtype=float32)

まぁ、良いんじゃないかな？  
これでy.data[i]でx[i]に対する解が見える。

In [43]:
f.cleargrads()

さっきは、適当に勾配gradsに0を代入してクリアしたが、本当はcleargrads()で勾配の累積を０にするみたい。  
ちなみにver1.15以前はzerograds()を使ってたみたいだけど、こっちの方が効率がよくて後方互換性のためだけに残してあるみたい

こちらの形式でも勾配を計算できる。やり方は同じで、初期誤差値を指定してからbackward()するだけ

In [44]:
y.grad = np.ones((3, 2), dtype=np.float32)
y.backward()
print("W grad:\n", f.W.grad)
print("b grad:\n", f.b.grad)

W grad:
 [[ 12.  15.  18.]
 [ 12.  15.  18.]]
b grad:
 [ 3.  3.]


もちろんここもy.grad dy/dw や y.grad y/dbになっている。(dy/dw = x, dy/db = 1 でそれぞれのｘデータに対して累積したものがgradに入るからあっているのが分かる）

# Write a model as Chain

やっとChainerらしくなってきた。  
ほとんどのNNは複数のlinksを含む。例えば多層パーセプトロンは複数の線形層で構成される。

In [45]:
l1 = L.Linear(4, 3)
l2 = L.Linear(3, 2)
def my_forward(x):
    h = l1(x)
    return l2(h)

なんとなく意味は分かる。４次元のxが入るとそれが２次元に変換されて出てきそうだ。  
しかしこの書き方は再利用が難しいのでダメ。もっとPythonっぽい書き方はクラスを使う方法だ。

In [46]:
class MyChain(Chain):
    def __init__(self):
        self.l1 = L.Linear(4, 3)
        self.l2 = L.Linear(3, 2)
        
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)

確かになんか再利用はしやすそうだ。いい具合な気がする。しかし、チュートリアル曰くもっと再利用しやすくするためにChainerクラスを上手く使うのが良いらしい。（先言えよ）

In [55]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1 = L.Linear(4, 3),
            l2 = L.Linear(3, 2),
        )
        
    def __call__(self, x):
        h = self.l1(x)
        j = self.l2(h)
        return j
a = MyChain()
print(a(np.array([[1,2,3,4]], dtype=np.float32)).data) # ちゃんと計算できることの確認

[[-2.85792923 -0.22626543]]


ここで"__call__"としているのはpythonっぽくさらにしようってことでしたみたい。これでインスタンを関数っぽく呼ぶだけで順方向の計算ができる

ここでは、MyChainのなかの要素としてl1, l2を定義したがこのようなlinksをMyChainの子linksと呼ぶ。  
Chainはlinksを継承しているので、子linksとしてChainを持てばより複雑な構造を持つことも可能となる。

他にもChainを定義する方法がある。ChainListを使う方法。　以下のようにlinksをリストのように扱える。  
self[]ってすげぇーな、おい。　びっくりしてるのは自分だけ？

In [57]:
class MyChain2(ChainList):
    def __init__(self):
        super(MyChain2, self).__init__(
            L.Linear(4, 3),
            L.Linear(3, 2),
        )
        
    def __call__(self, x):
        h = self[0](x)
        return self[1](h)
a = MyChain2()
print(a(np.array([[1,2,3,4]], dtype=np.float32)).data) # ちゃんと計算できることの確認

[[-0.76976579  0.02165219]]


もし子linksの名前がどうでも良いならChainListは有効だけど、逆に名前が決まっているならばChainが良い

# Optimizer